# MySQL AES_ENCRYPT / AES_DECRYPT
#### A python implementation of MySQL's AES_ENCRYPT/AES_DECRYPT functions with built-in MD5 key hashing

### Background
MySQL uses AES-128 encryption in ECB mode which allows SELECTing by encrypted values since no salt is used in this mode. Additionally, MySQL uses a particular algorithm to hash the given key before encrypting/decrypting. See http://www.maykinmedia.nl/blog/2012/nov/15/mysql-aes_encrypt-python/  

The MyCipher class below reproduces the functionality of the SQL AES encryption functions and includes built-in MD5 key hashing in order to mirror the recommended usage, as in the following SQL queries:
```sql
SELECT AES_ENCRYPT('secret value', MD5('encryption key') );
SELECT AES_DECRYPT(<ciphertext>, MD5('encryption key') );
```
Using MyCipher:
```python
MyCipher('encryption key').encrypt()
MyCipher('encryption key').decrypt(<ciphertext>)
```

##### note
Since ECB mode doesn't use IVs, it is important to salt the keys passed to MyCipher's constructor using a salt that will be know during lookup. e.g. A user ID could be prepended to the encryption key to ensure that each key is unique.

In [1]:
from Crypto.Cipher import AES
from Crypto.Hash import MD5
from binascii import hexlify, unhexlify

class MyCipher:
    def _setKey(self,key):
        key = MD5.new(key).hexdigest() # Hash key first
        # See http://www.maykinmedia.nl/blog/2012/nov/15/mysql-aes_encrypt-python/
        final_key = bytearray(16) # Start with 16-byte block of null characters.
        for i, c in enumerate(key): # Iterate over all characters in our key.
            final_key[i%16] ^= ord(c) # XOR the characters together.
        self.key = bytes(final_key) # Returns the resulting byte string.
        return self.key

    def _pad(self,s):
        pad_value = 16 - (len(s) % 16)
        return '%s%s' % (s, chr(pad_value)*pad_value)
    
    def _unpad(self,s):
        return s[:-ord(s[len(s)-1:])]
    
    def __init__(self,key):
        self._setKey(key)
        self.cipher=AES.new(self.key, AES.MODE_ECB)
        
    def encrypt(self,plaintext):
        return hexlify(self.cipher.encrypt(self._pad(plaintext)))
    
    def decrypt(self,ciphertext):
        return self._unpad(self.cipher.decrypt(unhexlify(ciphertext)))
    
    def getKey(self):
        return hexlify(self.key)
        

### Example usage

In [2]:
c = MyCipher('D476D59919BC')

print "encryption key: " + c.getKey()
print "cipher as hex: " + c.encrypt('foo')
encrypted = c.encrypt('foo')
print "cipher bytestring: " + unhexlify(encrypted)
print "decrypted: "+ c.decrypt(encrypted)

encryption key: 5d5b0005030f0108500a01540c570705
cipher as hex: 961d72fbde896818a97d6810c449fb90
cipher bytestring: �r�މh�}h�I��
decrypted: foo


## Verify encrypt/decrypt
### With MySQL:
```mysql
select aes_encrypt('foo', MD5('D476D59919BC')),
               hex(aes_encrypt('foo', MD5('D476D59919BC')));

+----------------------------------------+---------------------------------------------+
| aes_encrypt('foo',MD5('D476D59919BC')) | hex(aes_encrypt('foo',MD5('D476D59919BC'))) |
+----------------------------------------+---------------------------------------------+
| �r�މh�}h�I�                         | 961D72FBDE896818A97D6810C449FB90           |
+----------------------------------------+---------------------------------------------+```

```mysql
select aes_decrypt(X'961D72FBDE896818A97D6810C449FB90',MD5('D476D59919BC'));

+----------------------------------------------------------------------+
| aes_decrypt(X'961D72FBDE896818A97D6810C449FB90',MD5('D476D59919BC')) |
+----------------------------------------------------------------------+
| foo                                                                  |
+----------------------------------------------------------------------+```

### With openssl bash command:
See https://www.openssl.org/docs/manmaster/apps/enc.html
##### Encryption
```bash
echo -n "foo" | openssl aes-128-ecb -K 5d5b0005030f0108500a01540c570705
```
    �r�މh�}h�I�
Pipe that result to xxd to see hex-encoded cipher text.

```bash
echo -n "foo" | openssl aes-128-ecb -K 5d5b0005030f0108500a01540c570705 | xxd  -p
```
    961d72fbde896818a97d6810c449fb90

##### Decryption
To decode hex string, use:
```bash
echo -n 961D72FBDE896818A97D6810C449FB90 | xxd -r -p
```
    �r�މh�}h�I�

That can be piped to openssl:
```bash
echo -n 961D72FBDE896818A97D6810C449FB90 | xxd -r -p | openssl aes-128-ecb -d -K 5d5b0005030f0108500a01540c570705 
```
    foo